In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE #trains the model,determines the importance of each feature & then removes the weakest feature
from sklearn.linear_model import LogisticRegression # model
from sklearn.model_selection import train_test_split #training and testing
from sklearn.ensemble import RandomForestClassifier # reduces the risk of errors
from sklearn.preprocessing import StandardScaler #cleaning
from sklearn.metrics import accuracy_score # output score

In [17]:
def train_best_model():
    print("--- 1. Training Model using Historical Data ---")

    # Load dataset

    try:
        df = pd.read_csv('/content/HR_Data_Predict Employee Turnover.csv')
    except FileNotFoundError:
        print("ERROR: File not found. Please upload 'HR_Data_Predict Employee Turnover.csv' to Colab.")
        return None, None

    # Preprocessing (Must match the steps in employee_turnover_analysis.py)
    df['Department'] = np.where(df['Department'] == 'support', 'technical', df['Department'])
    df['Department'] = np.where(df['Department'] == 'IT', 'technical', df['Department'])
    df = pd.get_dummies(df, columns=['Department', 'salary'], drop_first=True)

    y = df['left']
    X = df.drop('left', axis=1).select_dtypes(include=np.number)

    # RFE Feature Selection
    log_model = LogisticRegression(max_iter=5000, random_state=0)
    rfe = RFE(estimator=log_model, n_features_to_select=10)
    rfe = rfe.fit(X, y)
    selected_features = X.columns[rfe.support_]

    X = X[selected_features]
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.3, random_state=0)

    # Train Random Forest (typically the best performer)
    rf_model = RandomForestClassifier(n_estimators=200, random_state=0)
    rf_model.fit(X_train, y_train)

    # We also fit the scaler, even though RF doesn't strictly need it,
    # as the new data prediction function might be reused for SVM later.
    scaler = StandardScaler()
    scaler.fit(X_train)

    print(f"Model trained using {len(selected_features)} features: {list(selected_features)}")
    return rf_model, scaler, list(selected_features)



In [18]:

# --- 2. Prediction Function ---

def preprocess_and_predict(model, scaler, selected_features, new_raw_data):
    """
    Preprocesses new raw data and makes predictions using the trained model.
    """
    print("\n--- 2. Preprocessing New Data and Predicting ---")

    # 1. Convert new data to DataFrame
    new_df = pd.DataFrame(new_raw_data)

    # Keep a copy of the original categorical values for the final output
    original_department = new_df['Department'].copy()
    original_salary = new_df['salary'].copy()

    # 2. Apply the SAME Department consolidation
    new_df['Department'] = new_df['Department'].replace(['support', 'IT'], 'technical')

    # 3. One-Hot Encode categorical features
    # This must be done on the full dataset column set to ensure all dummy columns are created
    new_df = pd.get_dummies(new_df, columns=['Department', 'salary'], drop_first=True)

    # 4. Align features with training features (CRUCIAL STEP)
    # Add missing dummy columns (and fill with 0) that existed in the training data but not in this small batch
    for feature in selected_features:
        if feature not in new_df.columns:
            new_df[feature] = 0

    # 5. Select and order ONLY the required features
    X_new = new_df[selected_features]

    # 6. Apply scaler (important if the final model was SVM or Logistic Regression)
    # For Random Forest, scaling doesn't change results, but it's good practice for reuse.
    X_new_scaled = scaler.transform(X_new)

    # 7. Make Predictions and get probabilities
    predictions = model.predict(X_new_scaled)
    # Probability of leaving (Class 1)
    probabilities = model.predict_proba(X_new_scaled)[:, 1]

    # 8. Format Results
    results_df = new_df.copy()
    results_df['Department'] = original_department
    results_df['salary'] = original_salary
    results_df['Prediction (0=Stay, 1=Left)'] = predictions
    results_df['Probability_of_Leaving'] = probabilities

    # Select key columns for final output
    return results_df[['satisfaction_level', 'last_evaluation', 'time_spend_company',
                       'Department', 'salary', 'Prediction (0=Stay, 1=Left)',
                       'Probability_of_Leaving']]

In [19]:

# --- 3. Execution ---

# A. Define New Employee Data
new_employees_data = [
    { # Employee A: Low Sat, High Hours, Long Tenure -> VERY HIGH RISK
        'satisfaction_level': 0.15, 'last_evaluation': 0.95, 'number_project': 6,
        'average_montly_hours': 280, 'time_spend_company': 5, 'Work_accident': 0,
        'promotion_last_5years': 0, 'Department': 'sales', 'salary': 'low'
    },
    { # Employee B: High Sat, Low Hours, Promoted -> LOW RISK
        'satisfaction_level': 0.85, 'last_evaluation': 0.50, 'number_project': 3,
        'average_montly_hours': 140, 'time_spend_company': 2, 'Work_accident': 1,
        'promotion_last_5years': 1, 'Department': 'accounting', 'salary': 'high'
    },
    { # Employee C: Testing 'support' department (should be technical) and medium risk
        'satisfaction_level': 0.40, 'last_evaluation': 0.70, 'number_project': 4,
        'average_montly_hours': 200, 'time_spend_company': 4, 'Work_accident': 0,
        'promotion_last_5years': 0, 'Department': 'support', 'salary': 'medium'
    }
]


# Train the model and get the necessary artifacts
rf_model, scaler, features = train_best_model()

if rf_model is not None:
    # Predict on the new data
    prediction_results = preprocess_and_predict(rf_model, scaler, features, new_employees_data)

    print("\n" + "="*50)
    print("FINAL PREDICTION RESULTS (Random Forest)")
    print("="*50)
    print(prediction_results)
    print("\n* 0 = Predicted to Stay, 1 = Predicted to Leave")

--- 1. Training Model using Historical Data ---


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=10 > n_features=7. There will be no feature selection and all features will be kept.
  warnings.warn(


Model trained using 7 features: ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']

--- 2. Preprocessing New Data and Predicting ---

FINAL PREDICTION RESULTS (Random Forest)
   satisfaction_level  last_evaluation  time_spend_company  Department  \
0                0.15             0.95                   5       sales   
1                0.85             0.50                   2  accounting   
2                0.40             0.70                   4     support   

   salary  Prediction (0=Stay, 1=Left)  Probability_of_Leaving  
0     low                            0                   0.410  
1    high                            0                   0.010  
2  medium                            0                   0.285  

* 0 = Predicted to Stay, 1 = Predicted to Leave


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
